## AutoReload `.py` files

In [1]:
%load_ext autoreload
%autoreload 2

## Определим необходимые константы

In [3]:
import os
import re
import sentencepiece as spm
from common.utils import (
    replace_numbers,
    create_sentencepiece_dataset
)


CODE_DIR = os.getcwd()
DATA_DIR = os.path.join(CODE_DIR, 'data')
CONLL2003_DIR = os.path.join(DATA_DIR, 'conll2003')
WNUT17_DIR = os.path.join(DATA_DIR, 'wnut17')
TOKENS_CONLL2003 = 30000
TOKENS_WNUT17 = 20000
TRAIN_SENTENCEPIECE_STRING = '--input={input_data} --model_prefix={prefix} --vocab_size={vocab_size} --model_type=bpe'

## Заменим все цифры на это слово [NUM]
`В загруженных на Github данных это уже было сделано, поэтому прогонять этот код необязательно`

In [3]:
for path in [
    os.path.join(CONLL2003_DIR, 'conll2003.train'),
    os.path.join(CONLL2003_DIR, 'conll2003.test'),
    os.path.join(CONLL2003_DIR, 'conll2003.valid'),
    os.path.join(WNUT17_DIR, 'wnut17conll.train'),
    os.path.join(WNUT17_DIR, 'wnut17conll.test')
]:
    txt = replace_numbers(path)
    with open(path, 'w', encoding='utf-8') as file:
        file.write(txt)

## Составим датасет для SentencePiece

In [4]:
paths = [
    os.path.join(CONLL2003_DIR, 'conll2003.train'),
    os.path.join(CONLL2003_DIR, 'conll2003.test'),
    os.path.join(CONLL2003_DIR, 'conll2003.valid'),
    os.path.join(WNUT17_DIR, 'wnut17conll.train'),
    os.path.join(WNUT17_DIR, 'wnut17conll.test')
]
save_paths = [f'{x}_sp' for x in paths]
for path, save_path in zip(paths, save_paths):
    create_sentencepiece_dataset(path, save_path)

### Теперь объединим `conll2003.train_sp`, `conll2003.test_sp` и `conll2003.valid_sp`

In [5]:
!cp ./data/conll2003/conll2003.train_sp ./data/conll2003/conll2003_sp_dataset.txt

!cat ./data/conll2003/conll2003.test_sp >> ./data/conll2003/conll2003_sp_dataset.txt
!cat ./data/conll2003/conll2003.valid_sp >> ./data/conll2003/conll2003_sp_dataset.txt

!rm ./data/conll2003/conll2003.train_sp ./data/conll2003/conll2003.test_sp ./data/conll2003/conll2003.valid_sp

In [6]:
spm.SentencePieceTrainer.Train(
    TRAIN_SENTENCEPIECE_STRING.format(
        input_data=os.path.join(CONLL2003_DIR, 'conll2003_sp_dataset.txt'),
        prefix='conll2003_sp',
        vocab_size=TOKENS_CONLL2003
    )
)

True

In [7]:
!python common/apply_sentencepiece.py \
    --dataset_paths ./data/conll2003/conll2003.train ./data/conll2003/conll2003.test ./data/conll2003/conll2003.valid \
    --sentencepiece_model_path conll2003_sp.model \
    --tagging_format IOB

2020-04-25 21:55:47.917 | INFO     | __main__:main:18 - Processing dataset at path: ./data/conll2003/conll2003.train
Processing lines: 28082it [00:02, 13817.41it/s]
2020-04-25 21:55:49.960 | SUCCESS  | __main__:main:27 - Successfully processed dataset and started saving at path: ./data/conll2003/conll2003.train.spm
2020-04-25 21:55:49.976 | SUCCESS  | __main__:main:31 - Saved dataset
2020-04-25 21:55:49.976 | INFO     | __main__:main:18 - Processing dataset at path: ./data/conll2003/conll2003.test
Processing lines: 6500it [00:00, 12878.53it/s]
2020-04-25 21:55:50.484 | SUCCESS  | __main__:main:27 - Successfully processed dataset and started saving at path: ./data/conll2003/conll2003.test.spm
2020-04-25 21:55:50.485 | SUCCESS  | __main__:main:31 - Saved dataset
2020-04-25 21:55:50.486 | INFO     | __main__:main:18 - Processing dataset at path: ./data/conll2003/conll2003.valid
Processing lines: 6906it [00:00, 16355.42it/s]
2020-04-25 21:55:50.909 | SUCCESS  | __main__:main:27 - Successfu

### Теперь объединим `wnut17conll.train_sp`, `wnut17conll.test_sp`

In [8]:
!cp ./data/wnut17/wnut17conll.train_sp ./data/wnut17/wnut17conll_sp_dataset.txt

!cat ./data/wnut17/wnut17conll.test_sp >> ./data/wnut17/wnut17conll_sp_dataset.txt

!rm ./data/wnut17/wnut17conll.train_sp ./data/wnut17/wnut17conll.test_sp

In [9]:
spm.SentencePieceTrainer.Train(
    TRAIN_SENTENCEPIECE_STRING.format(
        input_data=os.path.join(WNUT17_DIR, 'wnut17conll_sp_dataset.txt'),
        prefix='wnut17conll_sp',
        vocab_size=TOKENS_WNUT17
    )
)

True

In [10]:
!python common/apply_sentencepiece.py \
    --dataset_paths ./data/wnut17/wnut17conll.train ./data/wnut17/wnut17conll.test \
    --sentencepiece_model_path wnut17conll_sp.model \
    --tagging_format BIO

2020-04-25 21:55:54.279 | INFO     | __main__:main:18 - Processing dataset at path: ./data/wnut17/wnut17conll.train
Processing lines: 6788it [00:00, 12237.62it/s]
2020-04-25 21:55:54.838 | SUCCESS  | __main__:main:27 - Successfully processed dataset and started saving at path: ./data/wnut17/wnut17conll.train.spm
2020-04-25 21:55:54.840 | SUCCESS  | __main__:main:31 - Saved dataset
2020-04-25 21:55:54.840 | INFO     | __main__:main:18 - Processing dataset at path: ./data/wnut17/wnut17conll.test
Processing lines: 2574it [00:00, 10764.92it/s]
2020-04-25 21:55:55.080 | SUCCESS  | __main__:main:27 - Successfully processed dataset and started saving at path: ./data/wnut17/wnut17conll.test.spm
2020-04-25 21:55:55.081 | SUCCESS  | __main__:main:31 - Saved dataset
